In [ ]:
import json
import os
from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from copy import deepcopy
from collections import defaultdict

In [ ]:
def transform_to_view_frame(points, view_point, view_direction):
    shifted_points = points - view_point
    view_dir = view_direction / np.linalg.norm(view_direction)
    angle = -np.arctan2(view_dir[1], view_dir[0])
    rotation_matrix = np.array([
        [np.cos(angle), -np.sin(angle)],
        [np.sin(angle),  np.cos(angle)]
    ])
    return shifted_points @ rotation_matrix.T, rotation_matrix

In [ ]:
city_path = ... # Путь к объёмным файлам кварталов
with open(city_path, 'r', encoding='utf-8') as file:
    city2h = json.load(file)

In [ ]:
vidi = []
for k in view_files.keys():
    vidi += [len(view_files[k])]
vidi = np.array(vidi)

In [ ]:
view_files = {i[0]: [] for i in city2h}
data_path = r'E:\city_data'
for file in tqdm(os.listdir(data_path)):
    if 'json' in file:
        sc_name = '_'.join(file.split('_')[:-1])
        view_files[sc_name].append(file)

In [ ]:
def right_left_point(origin, direction, p1, p2):
    v1 = p1 - origin
    v2 = p2 - origin
    cross1 = direction[0]*v1[1] - direction[1]*v1[0]
    cross2 = direction[0]*v2[1] - direction[1]*v2[0]
    if cross1 < cross2:
        return 0, 1
    else:
        return 1, 0

def rotate_points(points, center, angle_deg):
    """Повернуть точки вокруг точки center на angle_deg (угол в градусах)."""
    angle_rad = np.deg2rad(angle_deg)
    # Смещаем в начало координат
    shifted = points - center
    # Матрица поворота
    rotation_matrix = np.array([
        [np.cos(angle_rad), -np.sin(angle_rad)],
        [np.sin(angle_rad),  np.cos(angle_rad)]
    ])
    # Поворачиваем и возвращаем обратно
    rotated = shifted @ rotation_matrix.T + center
    return rotated

def rotate_segments_to_above_point(segments, A):
    """Повернуть все отрезки на угол кратный 90 так, чтобы они оказались выше точки A, если возможно."""
    for angle in [0, 90, 180, 270]:
        rotated = rotate_points(segments.reshape(-1, 2), A, angle).reshape(-1, 2, 2)
        # Проверяем: все ли точки выше A по y
        if np.all(rotated[:, :, 1] > A[1]):
            return rotated, angle  # Успех: вернем повернутые отрезки и угол
    return None, None  # Не удалось

def hashable_point(pt):
    return tuple(pt)

def group_segments_to_polylines(segments):
    # Построим граф смежности
    adjacency = defaultdict(list)
    point_map = {}  # для восстановления координат
    for a, b in segments:
        a_key, b_key = hashable_point(a), hashable_point(b)
        adjacency[a_key].append(b_key)
        adjacency[b_key].append(a_key)
        point_map[a_key] = a
        point_map[b_key] = b

    visited_edges = set()
    polylines = []

    for a, b in segments:
        a_key, b_key = hashable_point(a), hashable_point(b)

        # Сортируем ключи, чтобы избежать дублирующего обхода
        edge_key = tuple(sorted([a_key, b_key]))
        if edge_key in visited_edges:
            continue
        visited_edges.add(edge_key)

        # Начинаем с [a, b], пробуем расширить
        line = [a_key, b_key]

        # Пробуем расширить вправо
        neighbors = adjacency[b_key]
        for c_key in neighbors:
            if c_key != a_key:
                line.append(c_key)
                break

        # Преобразуем в координаты
        polyline = [point_map[k] for k in line]
        polylines.append(np.array(polyline))

    return polylines

def bounding_box_features(points):
    min_xy = np.min(points, axis=0)
    max_xy = np.max(points, axis=0)
    center = (min_xy + max_xy) / 2
    size = max_xy - min_xy  # [width_x, height_y]
    return [center[0], center[1], size[0], size[1]]

In [ ]:
city_keys = {z[0]: i for i, z in enumerate(city2h)}

In [ ]:
res_path = ... # Путь для сохранения планарных видов

In [ ]:
props = 0
lost = 0
for ki in tqdm(all_scenes[1000:]):
    o_world = []
    #v_world = []
    rotates = []
    #t_points = []
    homes_list = city2h[city_keys[ki]][1]
    poly_shape = [len(i) for i in homes_list]
    if not(max(poly_shape)==5 and (min(poly_shape)==5)):
        #print('ПРОПУСК СЦЕНЫ')
        lost += 1
        continue
    homes = np.array(homes_list)
    his = city2h[city_keys[ki]][4]
    n_homes = len(homes)
    points = homes[:, :4, :]
    centroids = points.mean(axis=1)
    ranges = points.max(axis=1) - points.min(axis=1)
    #transformed = transform_to_view_frame(centroids, view_point, view_direction)
    bboxes = []
    visi = []
    total_boxes = []
    targets = []
    for nom, poly_home in enumerate(homes):
        position = [nom] + bounding_box_features(poly_home)
        total_boxes.append(position)
    total_boxes = np.array(total_boxes)
    for path in view_files[ki]:
        with open(data_path+'/'+path, 'r', encoding='utf-8') as file:
            view = json.load(file)
        o, v = np.array(view['o'])[:2], np.array(view['v'])[:2]
        homes_view = [int(w) for w in view['houses'].keys()]
        if len(homes_view) == 0:
            continue
        homes_sides = [list(view['houses'][str(n)].keys()) for n in homes_view]
        #transformed, R = transform_to_view_frame(centroids, o, v)
        o_world += [o]
        #v_world += [v]
        #rotates += [R]
        #t_points += [transformed]
        real_homes = [homes[j] for j in homes_view]
        orezki = []
        for hhh in real_homes:
            hm_otr = []
            for t in range(4):
                otr = np.array([hhh[t], hhh[t+1]])
                hm_otr.append(otr)
            orezki.append(hm_otr)
        orezki = np.array(orezki)
        otr_centrs = orezki.mean(axis=2)
        fin_otr = []
        lab_otr = []
        for ss, ha in enumerate(otr_centrs):
            distances = np.linalg.norm(ha - o, axis=1)
            closest_indices = np.argsort(distances)[:2]
            closest_centers = ha[closest_indices]
            r_id, l_id = right_left_point(o, v, closest_centers[0], closest_centers[1])
            closest_orezki = orezki[ss][closest_indices]
            storoni = homes_sides[ss]
            if 'right' in storoni:
                fin_otr.append(closest_orezki[r_id])
                lab_otr.append(homes_view[ss])
            if 'left' in storoni:
                fin_otr.append(closest_orezki[l_id])
                lab_otr.append(homes_view[ss])
        fin_otr = np.array(fin_otr)
        rotated_segments, used_angle = rotate_segments_to_above_point(fin_otr, o)
        if used_angle is None:
            #print(props, 'Не ориентируется над осью', ki)
            props += 1
            continue
        rotated_segments = rotated_segments- o
        otr_dict = {vv: [] for vv in homes_view}
        for segm, hos in zip(rotated_segments, lab_otr):
            otr_dict[hos].append(segm)
        att = False
        for k in otr_dict.keys():
            if len(otr_dict[k]) == 1:
                otr_dict[k] = otr_dict[k][0]
            else:
                if (otr_dict[k][0][1] == otr_dict[k][1][0]).all():
                    otr_dict[k] = np.array([otr_dict[k][0][0], otr_dict[k][0][1], otr_dict[k][1][1]])
                elif (otr_dict[k][0][0] == otr_dict[k][1][1]).all():
                    otr_dict[k] = np.array([otr_dict[k][0][1], otr_dict[k][0][0], otr_dict[k][1][0]])
                elif (otr_dict[k][0][1] == otr_dict[k][1][1]).all():
                    otr_dict[k] = np.array([otr_dict[k][0][0], otr_dict[k][0][1], otr_dict[k][1][0]])
                elif (otr_dict[k][1][0] == otr_dict[k][0][0]).all():
                    otr_dict[k] = np.array([otr_dict[k][0][1], otr_dict[k][0][0], otr_dict[k][1][1]])
                else:
                    att = True
        if att:
            #print(props, 'Не стыкуются сегменты', ki)
            props += 1
            continue
                    
        this_boxes = []
        for k in otr_dict.keys():
            position = [k] + bounding_box_features(otr_dict[k])
            this_boxes.append(position)
        this_boxes = np.array(this_boxes).astype(np.float16).tolist()
        bboxes.append(this_boxes)
        visi.append(int(path.split('.')[0].split('_')[-1]))
        rotates += [used_angle]
        
        alls_boxes = deepcopy(total_boxes)
        alls_boxes[:, 1:3] = rotate_points(alls_boxes[:, 1:3]-o, [0, 0], used_angle)
        if used_angle in [90, 270, -90]:
            alls_boxes[:, [3,4]] = alls_boxes[:, [4,3]]

        targets.append(alls_boxes.astype(np.float16).tolist())
        
        '''polylines = []
        for k in otr_dict.keys():
            polylines.append(otr_dict[k])
        fig, ax = plt.subplots(figsize=(6, 4))
        for cx, cy, dx, dy in alls_boxes[:, 1:]:
            x = cx - dx / 2
            y = cy - dy / 2
            rect = patches.Rectangle(
                (x, y), dx, dy,
                linewidth=1,
                edgecolor='black',
                facecolor='blue',
                alpha=0.3
            )
            ax.add_patch(rect)
        colors = ['r', 'b', 'g', 'orange', 'purple']
        for idx, polyline in enumerate(polylines):
            color = colors[idx % len(colors)]
            ax.plot(polyline[:, 0], polyline[:, 1], marker='o', color=color, label=f'Polyline {idx}')
            for i, (x, y) in enumerate(polyline):
                ax.text(x + 0.01, y + 0.01, str(i), fontsize=9, color=color)
        ax.set_aspect('equal')
        ax.grid(True)
        ax.legend()
        plt.show()'''
        
        #break
            
    camera = []
    for cp, rot_angle in zip(o_world, rotates):
        centerd_o = o_world - cp
        oriented = rotate_points(centerd_o, [0, 0], rot_angle)
        camera += [oriented]
    camera = np.array(camera).astype(np.float16).tolist()

    record_dict = {}
    record_dict['n'] = n_homes
    record_dict['v'] = visi
    record_dict['c'] = camera
    record_dict['b'] = bboxes
    record_dict['t'] = targets

    with open(f"{res_path}/{ki}.json", "w") as f:
        json.dump(record_dict, f)

    #break